In [ ]:
#Put these files in the same directory as this notebook
#Run this cell first

%runfile fd_wireframe.sage
%runfile KBesselPrecomputer.sage

# Basic Demo of a Blank Fundamental Domain

In [ ]:
# fd_wireframe is the function that does everything. Here is its signature.
'''
def fd_wireframe(d, verbose=0, opacity=1, thickness=2,
                 plot_function=False,
                 f=0, plane_axis="x", plane_value="0",
                 animate=False,
                 values_list=[],
                 zmax=3,
                 frame=False,
                 function_plot_points=200,
                 function_colormap = None,
                 #browser="/usr/bin/firefox"):
                 browser="/opt/google/chrome/chrome"):
'''

#Example usage
#Only use d in [19, 43, 67, 163]
#Any other number may or may not work, no guarantees

fd_plot = fd_wireframe(19, opacity=0.3, thickness = 0.5)

#To make the floor of the fundamental domain opaque, run this code
#fd_plot = fd_wireframe(43, opacity=1)

#To make the outline thinner or thicker, pass in your preferred thickness parameter
#fd_plot = fd_wireframe(43, opacity=0.3, thickness=1)

#To make the top of the fundamental domain extend higher or be cut off lower, pass in a zmax parameter
#fd_plot = fd_wireframe(43, opacity=0.3, zmax=10)







#Leave this uncommented to see any of the plots above
show(fd_plot, frame=True, axes=False)


# How to Plot a Function Along a Plane

In [ ]:
#Here is a random, easy function that we will plot

import numpy as np

def demo_function(x0, x1, y):
    return x0**2 - x1**2 + np.cos(2.0 * np.pi*y)

#We have to tell fd_wireframe how to color the outputs of demo_function
#Color space functions have some fixed interval as their input, so we have to map the output of demo_function to that fixed interval

#Here is one way to do this.
#Assume the color space function assumes inputs in the interval [0,1]
#If the function has image [a,b] and a < 0 < b (very typical), then I will compose with the linear function that
#sends max(|a|, |b|) to 1, -max(|a|, |b|) to 0, and 0 to 1/2
#In order to know this interval [a,b], we simply compute the function at many points in the domain we would like to plot it.

#Let's plot the function in the fundamental domain for d = 19 with maximum vertical height of 5

d = 19 
A = np.float64(sqrt(d)/2)
Y0 = np.float64(sqrt(2/d))

mesh = np.float64(0.01)

x0min = -0.5
x0max = 0.5
x1min = -A/2
x1max = A/2
ymin = Y0
ymax = 5

f_max = np.float64(0)
f_min = np.float64(0)

for x0 in np.linspace(x0min, x0max, int((x0max - x0min) / mesh + 1)).tolist():
    for x1 in np.linspace(-A/2, A/2, int((x1max - x1min) / mesh + 1)).tolist():
        for y in np.linspace(Y0, ymax, int((ymax - ymin) / mesh + 1)).tolist():
            f_val = demo_function(x0, x1, y)
            if f_val > f_max:
                f_max = f_val
            if f_val < f_min:
                f_min = f_val

print("Function max is: " + str(f_max), "Function min is: " + str(f_min))

m = max(np.abs(f_max), np.abs(f_min))

#Define a helper function that normalizes the actual function we want to plot, implementing the linear map discussed earlier

def normalized_demo(x0,x1,y):
    return demo_function(x0,x1,y)/(2.0*m) + 0.5


#Now pass this function in to fd_wireframe in the following way

fd_plot_1 = fd_wireframe(d, 
                       verbose=0, 
                       opacity=0.3, 
                       thickness=0.5,
                       plot_function=True,
                       f=normalized_demo,
                       plane_value=0,
                       zmax=ymax,
                       animate=False,
                       function_plot_points=200,
                       plane_axis="x")
show(fd_plot_1)

fd_plot_2 = fd_wireframe(d, 
                       verbose=0, 
                       opacity=0.3, 
                       thickness=0.5,
                       plot_function=True,
                       f=normalized_demo,
                       plane_value=0,
                       zmax=ymax,
                       animate=False,
                       function_plot_points=200,
                       plane_axis="y")

show(fd_plot_2)


# How to Make an Animation

In [ ]:
#The only real difference is you tell the function that you're making an animation and pass in a list
#of values that describe the plane. For example

num_frames = 100
values_list = np.linspace(x0min, x0max, num_frames).tolist()
print(values_list)

#This saves rendered images to a folder. These images can be strung together into an animated GIF. I recommend using a tool like
#ffmpeg used in the terminal.
#I get good quality results with
'''
 ffmpeg \
  -framerate 30 \
  -pattern_type glob \
  -i '*.png' \
  -loop 0 \
  -filter_complex "[0]reverse[r];[0][r]concat=n=2:v=1:a=0,split[s0][s1];[s0]palettegen[p];[s1][p]paletteuse"  \
  out.gif
'''

fd_plot = fd_wireframe(d, 
                       verbose=0, 
                       opacity=0.3, 
                       thickness=0.5,
                       plot_function=True,
                       f=normalized_demo,
                       zmax=ymax,
                       animate=True,
                       plane_axis="x", 
                       values_list=values_list)

# How to Plot a Bianchi-Maass Form

Essentially, all we do is load in the Bianchi-Maass form data from a file and that is our function. The previous ideas apply.

In [ ]:
import re
import numpy as np
CBF is ComplexBallField() is ComplexBallField(128)
RR = RealField(54)
CC = ComplexField(54)

d = 19
symClass = 'D'
theta = np.complex128((1 + I*sqrt(d))/2)
A = theta.imag
Y0 = np.sqrt(2.0/d)

import os
import glob

def get_sorted_filenames_by_value(folder_path):
    # Get all .txt files in the folder
    txt_files = glob.glob(os.path.join(folder_path, "*.txt"))
    
    # Extract filenames without extension and convert to float
    files_with_values = []
    for file in txt_files:
        filename = os.path.splitext(os.path.basename(file))[0]
        try:
            value = float(filename)
            files_with_values.append((filename, value))
        except ValueError:
            continue 
    
    # Sort by the floating-point value
    files_with_values.sort(key=lambda x: x[1])
    
    # Return only the sorted filenames
    return [f[0] for f in files_with_values]

#Replace this with where you have the files stored
directory = "/Users/ericmoss/CLionProjects/bianchi-maass-forms/Output/Tested, Hand Checked, and Final/"
directory += str(d) + "/"
directory += str(symClass) + "/"

params = get_sorted_filenames_by_value(directory)
param_index = -1

#We will show this for the largest parameter, the user may change this to whichever parameter they wish
r_spec_param = RR(params[param_index])
print("d =", d, "symClass =", symClass, "r =", r_spec_param)
nu = CBF(0,r_spec_param)

file = open(directory + params[param_index] + ".txt")

lines = []
for read_in in file:
    if read_in[0] not in "0123456789":
        continue
    line = re.split(",|\n", read_in)
    line = [itr for itr in line if itr != ""]
    line = [RR(c) for c in line]
    if len(line) == 3:
        line = [int(line[0]), int(line[1]), line[2]]
        lines.append(line)

dtype = np.dtype([('index_abs', np.float64), ('index_real', np.float64), ('index_imag', np.float64), ('fourier_coeff', np.float64)])

indices_and_coeffs = np.array([], dtype=dtype)
for line in lines:
    m = np.cdouble(line[0] + 0.5 * line[1] + I*(line[1] * A))
    a = np.float64(line[2])
    new_element = np.array([(np.abs(m), m.real, m.imag, a)], dtype=dtype)
    indices_and_coeffs = np.append(indices_and_coeffs, new_element)

indices_and_coeffs_no_conj = np.array([], dtype=dtype)
indices_and_coeffs_yes_conj = np.array([], dtype=dtype)

for itr in indices_and_coeffs:
    if itr[1] == 0 or itr[2] == 0:
        indices_and_coeffs_no_conj = np.append(indices_and_coeffs_no_conj, itr)
    else:
        indices_and_coeffs_yes_conj = np.append(indices_and_coeffs_yes_conj, itr)

## Run this cell to initialize a fast K-Bessel linear interpolation (required)

In [ ]:
Kb = KBesselPrecomputer(r_spec_param, 2.0*np.pi/A * 1.0 * Y0)

## The next cell defines the Bianchi-Maass form from the coefficients

In [ ]:
twoPiOverA = np.float64(2*np.pi/A)

#The argument of sin/cos is 2*pi/(2*A) * <l, ix> which we can optimize slightly to be -2*pi/A (x1*Re*(l) + Im(l)*x0)
trigArgScale = np.float64(-2*np.pi/A)

def f(x0, x1, y):
    answer = np.float64(0)
    x0 = np.float64(x0)
    x1 = np.float64(x1)
    y = np.float64(y)

    if symClass == 'D' or symClass == 'G':
        for itr in indices_and_coeffs_no_conj:
            bess_arg = twoPiOverA * itr[0] * y
            bess_val = Kb.bess(bess_arg)
            if bess_val < 0.000001 and bess_arg > r_spec_param:
                break
            term = np.float64(0)
            term = itr[3] * bess_val * np.cos(trigArgScale * (itr[1]*x1 + itr[2]*x0))
            answer += term

        for itr in indices_and_coeffs_yes_conj:
            bess_arg = twoPiOverA * itr[0] * y
            bess_val = Kb.bess(bess_arg)
            if bess_val < 0.000001 and bess_arg > r_spec_param:
                break
            coeff_bess = itr[3] * bess_val
            term = coeff_bess * (np.cos(trigArgScale * (itr[1]*x1 + itr[2]*x0)) + np.cos(trigArgScale * (-itr[1]*x1 + itr[2]*x0)))
            answer += term
    else:
        for itr in indices_and_coeffs_no_conj:
            bess_arg = twoPiOverA * itr[0] * y
            bess_val = Kb.bess(bess_arg)
            if bess_val < 0.000001 and bess_arg > r_spec_param:
                break
            term = np.float64(0)
            term = itr[3] * bess_val * np.sin(trigArgScale * (itr[1]*x1 + itr[2]*x0))
            answer += term

        for itr in indices_and_coeffs_yes_conj:
            bess_arg = twoPiOverA * itr[0] * y
            bess_val = Kb.bess(bess_arg)
            if bess_val < 0.000001 and bess_arg > r_spec_param:
                break
            coeff_bess = itr[3] * bess_val
            term = coeff_bess * (np.sin(trigArgScale * (itr[1]*x1 + itr[2]*x0)) + np.sin(trigArgScale * (-itr[1]*x1 + itr[2]*x0)))
            answer += term
    

    return y * answer * 2
    

## Determine the max and min outputs of the form

In [ ]:
x0min = -0.5
x0max = 0.5
x1min = -A/2
x1max = A/2
ymin = Y0
ymax = 5

mesh = np.float64(0.1)

f_max = np.float64(0)
f_min = np.float64(0)

for x0 in np.linspace(x0min, x0max, int((x0max - x0min) / mesh + 1)).tolist():
    for x1 in np.linspace(-A/2, A/2, int((x1max - x1min) / mesh + 1)).tolist():
        for y in np.linspace(Y0, ymax, int((ymax - ymin) / mesh + 1)).tolist():
            f_val = f(x0, x1, y)
            if f_val > f_max:
                f_max = f_val
            if f_val < f_min:
                f_min = f_val

print("Function max is: " + str(f_max), "Function min is: " + str(f_min))

scale = np.float64(2.0* max(np.abs(f_max), np.abs(f_min)))

#Define a helper function that normalizes the actual function we want to plot, implementing the linear map discussed earlier

def normalized_f(x0,x1,y):
    return f(x0,x1,y)/scale + np.float64(0.5)

## Render a test plot

In [ ]:
from sage.plot.contour_plot import ContourPlot
x0,x1,y=var('x0,x1,y')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sage.all import *

x0 = 0
cplot = contour_plot(lambda x1,y : normalized_f(0.5,x1,y), 
                     (x1,-A/2,A/2), 
                     (y, Y0, ymax), 
                     plot_points=100, 
                     aspect_ratio=1, 
                     contours = 256, 
                     cmap = colormaps.PuOr)
show(cplot)

## Render a single frame

In [ ]:
bm_plot = fd_wireframe(d, 
                       verbose=0, 
                       opacity=0.3, 
                       thickness=0.5,
                       plot_function=True,
                       f=normalized_f,
                       plane_value=0,
                       zmax=ymax,
                       animate=False,
                       function_plot_points=200,
                       function_colormap = colormaps.PuOr,
                       plane_axis="x")
#show(bm_plot)

## Render the frames for an animation of plots of a Bianchi-Maass form

In [ ]:
num_frames = 100
values_list = np.linspace(x0max, x0min, num_frames).tolist()
print(values_list)

#This saves rendered images to a folder. These images can be strung together into an animated GIF. I recommend using a tool like
#ffmpeg used in the terminal.
#I get good quality results with
'''
 ffmpeg \
  -framerate 30 \
  -pattern_type glob \
  -i '*.png' \
  -loop 0 \
  -filter_complex "[0]reverse[r];[0][r]concat=n=2:v=1:a=0,split[s0][s1];[s0]palettegen[p];[s1][p]paletteuse"  \
  out.gif
'''

fd_plot = fd_wireframe(d, 
                       verbose=0, 
                       opacity=0.1, 
                       thickness=0.2,
                       plot_function=True,
                       f=normalized_f,
                       zmax=ymax,
                       animate=True,
                       plane_axis="x", 
                       function_colormap = colormaps.PuOr,
                       function_plot_points = 200,
                       values_list=values_list)

There are options you can tweak in the code of "fd_wireframe" as well. Look for the line

"plot_this_plane.save(...)"

There you can make changes such as where the camera is positioned, what point the camera is looking at, render quality, etc.

Calling "tachyon_rt.usage()" in this notebook also gives more options you can pass via the extra_opts string.